# Pydantic Schema
*A Pydantic schema is a Python object and cannot be sent directly to PostgreSQL. Data must be translated into SQL parameters. SQL defines the operation, parameters carry the values, and tuples ensure safe and ordered execution in .execute().*

# 1️⃣ If we send product: ProductSchema to the database — what is it?
#### Does it work? Why / why not?

ProductSchema (for example, a Pydantic model) is a Python object.

That means:
- It lives in Python memory
- It contains attributes and types
- It is not something a database understands
- PostgreSQL cannot store Python objects directly.

#### Does it work?

❌ No, not by itself.

PostgreSQL understands:
- SQL statements
- Primitive values (INTEGER, TEXT, NUMERIC, JSONB, etc.)

So before sending data to the database, we must extract values from the object.

#### What actually works

You must convert the object into primitive values, for example:
- product.name
- product.price
- product.category

This is part of data translation, which is central to ETL pipelines.

# 2️⃣ What is SQL, and what are parameters?
#### What is SQL?
SQL (Structured Query Language) is the language used to:
- Insert data
- Read data
- Update data
- Delete data

PostgreSQL only understands SQL, not Python code.

#### What are parameters?
Parameters are placeholders inside SQL queries.

Instead of putting values directly into the query, you write:

```INSERT INTO products (name, price)
VALUES (%s, %s);
```
Then you provide the actual values separately.

#### Why this matters:
- Prevents SQL injection
- Handles data types safely
- Lets the database optimize queries

This is how Python safely talks to PostgreSQL using drivers like
psycopg.

# 3️⃣ Why are tuples important? (.execute())

In psycopg, parameters are passed as a tuple.
**Tuples are the contract that binds Python values to SQL parameters.**

In [ ]:
# sample
cursor.execute(
    "INSERT INTO products (name, price) VALUES (%s, %s)",
    ("Laptop", 1299.00)
)

#### Why a tuple?
- Tuples are ordered
- Tuples are immutable
- The order must match the placeholders in SQL

#### If you don’t use a tuple:
- The query may fail
- Values may be misaligned
- Safety guarantees are lost

In [ ]:
Python object (ProductSchema)
        ↓ extract fields
Primitive values (str, int, float)
        ↓ parameterized SQL
SQL + tuple
        ↓
PostgreSQL
